In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rnd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('../input/titanic/train.csv')
test_df = pd.read_csv('../input/titanic/test.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.tail()

In [ ]:
train_df.info()
print('_'*40)
test_df.info()

In [ ]:
print(train_df.columns.values)

print([col for col in train_df.columns if train_df[col].dtype=='object' and train_df[col].nunique()<10])

print([col for col in train_df.columns if train_df[col].dtype in ['int64','float64']])

In [ ]:
train_df.describe()

In [ ]:
train_df.describe(include=['O'])

In [ ]:
train_df[['Pclass','Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)
# Pclass 1 had higher survival percentage

In [ ]:
train_df[['Sex','Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)
# Females had higher survival percentage

In [ ]:
train_df[['Embarked','Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[['Parch','Survived']].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_df[['SibSp','Survived']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
# Corelating numerical features
grid = sns.FacetGrid(train_df, col='Survived')
grid.map(plt.hist, 'Age', bins=20)

In [ ]:
print("Distribution of Survived==0")
print(train_df.loc[train_df.Survived==0].Sex.value_counts())
print("\n")
print("Distribution of Survived==1")
print(train_df.loc[train_df.Survived==1].Sex.value_counts())

In [ ]:
# Corelating numerical and categorical features
grid = sns.FacetGrid(train_df, col='Survived', row='Pclass', height=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=0.5, bins=20)
grid.add_legend()

In [ ]:
# Corelating numerical and categorical features
grid = sns.FacetGrid(train_df, col='Survived', row='Sex', height=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=0.5, bins=20)
grid.add_legend()

In [ ]:
# Corelating categorical features
grid = sns.FacetGrid(train_df, row='Embarked', height=2.2, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep', order=[1,2,3], hue_order=['female','male'])
grid.add_legend()

In [ ]:
print(train_df.loc[(train_df.Sex=='male') & (train_df.Embarked=='C') & (train_df.Pclass==1)].Survived.mean())
print(train_df.loc[(train_df.Sex=='male') & (train_df.Embarked=='C') & (train_df.Pclass==2)].Survived.mean())
print(train_df.loc[(train_df.Sex=='male') & (train_df.Embarked=='C') & (train_df.Pclass==3)].Survived.mean())
print(train_df.loc[(train_df.Sex=='female') & (train_df.Embarked=='C') & (train_df.Pclass==1)].Survived.mean())
print(train_df.loc[(train_df.Sex=='female') & (train_df.Embarked=='C') & (train_df.Pclass==2)].Survived.mean())
print(train_df.loc[(train_df.Sex=='female') & (train_df.Embarked=='C') & (train_df.Pclass==3)].Survived.mean())


In [ ]:
# Correlating categorical features and numerical features
grid = sns.FacetGrid(train_df, col='Survived', row='Embarked', height=2.2, aspect=1.6)
grid.map(sns.barplot, 'Sex', 'Fare', order=['female','male'], alpha=0.5, ci=None)
grid.add_legend()

In [ ]:
print(train_df.loc[(train_df.Sex=='female') & (train_df.Embarked=='C') & (train_df.Survived==1)].Fare.mean())
print(train_df.loc[(train_df.Sex=='male') & (train_df.Embarked=='C') & (train_df.Survived==1)].Fare.mean())

In [ ]:
print("Before", train_df.shape, test_df.shape)

train_df.columns
train_df = train_df.drop(['Ticket','Cabin'], axis=1)
test_df = test_df.drop(['Ticket','Cabin'], axis=1)
combine = [train_df, test_df]

print("After", train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

In [ ]:
train_df.head()

In [ ]:
# Creating new feature from existing
for dataset in combine:
    print(dataset.shape)
    dataset['Title'] = dataset.Name.str.extract('([A-Za-z]+)\.', expand=False)

pd.crosstab(train_df.Title, train_df.Sex)

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady','Countess','Capt', 'Col',
                                'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

print(train_df.groupby('Title').Survived.mean())

train_df[['Title','Survived']].groupby('Title', as_index=False).mean()

In [ ]:
title_mapping = {"Mr":1, "Miss":2, "Mrs":3, "Master":4, "Rare":5}

for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping).astype(int)
    dataset['Title'] = dataset['Title'].fillna(0)
    
train_df.head()

In [ ]:
# Drop the Name and PassengerId
train_df = train_df.drop(['Name','PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]

In [ ]:
train_df.head()

In [ ]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map({"male":0, "female":1}).astype(int)

train_df.head()

In [ ]:
train_df.info()

In [ ]:
# Completing a numerical continuous feature - Age
grid = sns.FacetGrid(train_df, row='Pclass', col='Sex', height=2.2, aspect=1.6)
grid.map(plt.hist, 'Age', bins=20)
grid.add_legend()

In [ ]:
guess_ages = np.zeros((2,3))
guess_ages

In [ ]:
for dataset in combine:
    for i in range(0,2):
        for j in range(0,3):
            guess_df = dataset[(dataset.Sex==i) & (dataset.Pclass==j+1)].Age.dropna()
            print(guess_df)
            age_guess = guess_df.median()
            
            guess_ages[i,j] = int(age_guess/.5 + 0.5) * 0.5

In [ ]:
print(guess_ages)

In [ ]:
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex==i) & (dataset.Pclass==j+1), 'Age'] = guess_ages[i,j]
    dataset['Age'] = dataset['Age'].astype(int)
    
train_df.info()

In [ ]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand','Survived']].groupby('AgeBand', as_index=False).mean().sort_values(by='AgeBand', ascending=True)

In [ ]:
for dataset in combine:
    dataset.loc[(dataset.Age <= 16), 'Age'] = 0
    dataset.loc[(dataset.Age > 16) & (dataset.Age <= 32), 'Age'] = 1
    dataset.loc[(dataset.Age > 32) & (dataset.Age <= 48), 'Age'] = 2
    dataset.loc[(dataset.Age > 48) & (dataset.Age <= 64), 'Age'] = 3
    dataset.loc[(dataset.Age > 64), 'Age'] = 4
    
    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()

In [ ]:
train_df = train_df.drop('AgeBand', axis=1)
combine = [train_df, test_df]

In [ ]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df[['FamilySize','Survived']].groupby('FamilySize', as_index=False).mean().sort_values(by='FamilySize', ascending=True)

In [ ]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[(dataset.FamilySize==1), 'IsAlone'] = 1

train_df[['IsAlone','Survived']].groupby('IsAlone', as_index=False).mean()

In [ ]:
train_df = train_df.drop(['Parch','SibSp','FamilySize'], axis=1)
test_df = test_df.drop(['Parch','SibSp','FamilySize'], axis=1)
combine=[train_df, test_df]

train_df.head()

In [ ]:
freq_port = train_df.Embarked.dropna().mode()[0]
print(freq_port)

In [ ]:
for dataset in combine:
    dataset.Embarked = dataset.Embarked.fillna(freq_port)
    
train_df[['Embarked','Survived']].groupby('Embarked', as_index=False).mean().sort_values(by='Survived', ascending=True)

In [ ]:
for dataset in combine:
    dataset.Embarked = dataset.Embarked.map({"S":0, "C":1, "Q":2}).astype(int)

train_df.head()

In [ ]:
# Quick completing and converting a numeric feature
test_df.info()

In [ ]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.info()

In [ ]:
# Create Fare Band
train_df['FareBand'] = pd.qcut(train_df.Fare, 4)
train_df[['FareBand','Survived']].groupby('FareBand', as_index=False).mean().sort_values(by='FareBand', ascending=True)

In [ ]:
for dataset in combine:
    dataset.loc[(dataset.Fare <= 7.91), 'Fare'] = 0
    dataset.loc[(dataset.Fare > 7.91) & (dataset.Fare <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset.Fare > 14.454) & (dataset.Fare <= 31.0), 'Fare'] = 2
    dataset.loc[(dataset.Fare > 31.0), 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop('FareBand', axis=1)
combine=[train_df, test_df]

In [ ]:
train_df.head(10)

In [ ]:
X_train = train_df.drop('Survived', axis=1)
Y_train = train_df.Survived
X_test = test_df.drop('PassengerId', axis=1)
X_train.shape, Y_train.shape, X_test.shape

In [ ]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

In [ ]:
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

coeff_df.sort_values(by='Correlation', ascending=False)

In [ ]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

In [ ]:
# Gaussian Naive Bayes

gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

In [ ]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

In [ ]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

In [ ]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

In [ ]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

In [ ]:
# Random Forest
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)

acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)